# Conventional implicatures

In this notebook we will explore how to construct a lexicon that includes expressions conveying some form of conventional implicature. Let's start right away with a simple example involving an expletive like *fucking*. Let's see how we can get an analysis for a simple sentence such as *John loves fucking McDonald's*. As usual we start by importing the necessary theorem prover code:


In [ ]:
from mb.tp import *

Then we are going to define a simple set of resources. *John*, *McDonald's* and *loves* will have be represented with resources having their standard semantic types that we are used to:

In [ ]:
e_type = AtomicType('e')
t_type = AtomicType('t')
john = Atom('j', e_type)
mcdonalds = Atom('m', e_type)
loves = mcdonalds ** john ** Atom('l', t_type)

The expletive *fucking* will be represented by a resource of type $e \rightarrow \lozenge e$, basically attaching to the sentence a side-comment related to its argument (McDonald's) and returning its argument untouched for the at-issue component of meaning:

In [ ]:
fucking = mcdonalds ** Monad(mcdonalds, 'ci')
goal = Monad(Atom('l', t_type), 'ci')

``goal`` is the resource corresponding to the meaning of a sentence with both the at-issue and the side-issue material. Here we use the ``ci`` modality. While it's not important in this case, when we will analyse different types of side-effecting expressions interacting this will allow us to separate the various semantic contributions. Now we can ask the theorem prover to construct the readings (in this case a single one) associated with the sentence:

In [ ]:
from mb.lambda_calc import Const
s = Sequent([john, loves, fucking, mcdonalds], goal)
proofs = prove(s, [Const('john'), Const('love'), Const('fucking'), Const('McD')])
proofs.group_proofs()
proofs.show(True)

Next let us consider a slightly more complicated example with an appositive: *John Lee Hooker, the bluesman from Tennessee, appeared in The Blues Brothers*. For this example we are going to use the categorial grammar parser by defining the appropriate syntactic types and binding them to the meanings using a ``Lexicon`` object. First we import the necessary material:

In [ ]:
from mb.cg import *
from mb.lambda_calc import *

And then we define the syntactic types and the lexicon:

In [ ]:
np = Atom('np', AtomicType('e'))
s = Atom('s', AtomicType('t'))
n = Atom('n', AtomicType('n'))
lexicon = Lexicon()
lexicon.add_entry('JLH', np, Const('jlh'))
lexicon.add_entry('the', (np >> s) << n, Var('x') ^ Var('x'))
lexicon.add_entry('bluesman', n, Const('bluesman'))
lexicon.add_entry('from', (n >> n) << np, Var('x') ^ (Var('P') ^ (Var('y') ^ (Const('AND')(Var('P')(Var('y')))(Const('from')(Var('y'))(Var('x'))))))) 
lexicon.add_entry('Tennessee', np, Const('Tennessee'))
lexicon.add_entry('appeared_in', (np >> s) << np, Var('x') ^ (Var('y') ^ (Const('appeared_in')(Var('y'))(Var('x')))))
lexicon.add_entry('TBB', np, Const('tbb'))
lexicon.add_entry(',', (np >> Monad(np, 'ci')) << (np >> s), Var('P') ^ (Var ('x') ^ ((Const('WRITE')(Var('P')(Var('x')))) ** (Var('y') ^ Unit(Var('x'))))))

Now we can do a proof for the sentence (notice that we have to make some orthographical modifications to make it palatable for the parser, such as having just one comma and joining *appeared in* into a single word token):

In [ ]:
proofs = prove_sentence('JLH , the bluesman from Tennessee appeared_in TBB', Monad(s, 'ci'), lexicon)
proofs.show(True)

(Notice that the proof is quite wide so you may have to scroll right to admire it in its entirety).